# Metaheuristics TP7
Ning, Tientso

In [70]:
import numpy as np
import random
import math

In [2]:
# This is the machine on which programs are executed
# The output is the value on top of the pile. 
class CPU:
    def __init__(self):
        self.pile= []
    def reset(self):
        while len(self.pile)>0:self.pile.pop()

In [71]:
# These are the instructions
def AND(cpu, data):
    try:
        v1 = cpu.pile.pop()
        v2 = cpu.pile.pop()
        cpu.pile.append(v1 and v2) #logical AND
    except:
        pass

def OR(cpu, data):
    try:
        v1 = cpu.pile.pop()
        v2 = cpu.pile.pop()
        cpu.pile.append(v1 or v2) #logical OR
    except:
        pass
    
def XOR(cpu, data):
    try:
        v1 = cpu.pile.pop()
        v2 = cpu.pile.pop()
        cpu.pile.append(v1 ^ v2) #logical XOR
    except:
        pass

def NOT(cpu, data):
    try:
        v = cpu.pile.pop()
        cpu.pile.append(not v) #logical NOT
    except:
        pass
    
# Push values of variables on the stack.      
def X1(cpu, data):
    cpu.pile.append(data[0])
def X2(cpu, data):
    cpu.pile.append(data[1])
def X3(cpu, data):
    cpu.pile.append(data[2])
def X4(cpu, data):
    cpu.pile.append(data[3])
    
# Execute a program
def execute(program,cpu, data):
    
    #check each instruction in our program
    for instruction in program:
        if instruction == "AND":
            AND(cpu, data)
        elif instruction == "OR":
            OR(cpu, data)
        elif instruction == "XOR":
            XOR(cpu, data)
        elif instruction == "NOT":
            NOT(cpu, data)
        elif instruction == "X1":
            X1(cpu, data)
        elif instruction == "X2":
            X2(cpu, data)
        elif instruction == "X3":
            X3(cpu, data)
        elif instruction == "X4":
            X4(cpu, data)
    try:
        return cpu.pile.pop() #top item in the stack is your answer
    except:
        return None

# Generate a random program
def randomProg(length,functionSet,terminalSet):
    
    program = []
    
    #generate a program of length length
    for i in range(length):
        
        #choose from functionSet or terminalSet, "randomly"
        if math.floor(random.random() + 0.5) < 1.0:
            program.append(functionSet[random.randint(0, len(functionSet)-1)])
        else:
            program.append(terminalSet[random.randint(0, len(terminalSet)-1)])

    return program

# Computes the fitness of a program. 
# The fitness counts how many instances of data in dataSet are correctly computed by the program
def computeFitness(prog,cpu,dataSet): 
    
    fitness = 0
    
    for i in range(0, len(dataSet)): #for each row of dataSet
        row = dataSet[i]
        if execute(prog,cpu, row) == row[len(row)-1]: #last entry in row is the intended value
            fitness += 1
    
    return fitness

In [72]:
# Selection using 2-tournament.
def selection(Population,cpu,dataSet):
    listOfFitness=[]
    for i in range(len(Population)):
        prog=Population[i]
        f=computeFitness(prog,cpu,dataSet)
        listOfFitness.append( (i,f) )

    newPopulation=[]
    n=len(Population)
    for i in range(n):    
        i1=random.randint(0,n-1)
        i2=random.randint(0,n-1)
        if listOfFitness[i1][1]>listOfFitness[i2][1]:
            newPopulation.append(Population[i1])
        else:
            newPopulation.append(Population[i2])
    return newPopulation

def crossover(Population,p_c):
    newPopulation=[]
    n=len(Population)
    i=0
    while(i<n):
        p1=Population[i]
        p2=Population[(i+1)%n]
        m=len(p1)
        if random.random()<p_c:  # crossover
            k=random.randint(1,m-1)
            newP1=p1[0:k]+p2[k:m]
            newP2=p2[0:k]+p1[k:m]
            p1=newP1
            p2=newP2
        newPopulation.append(p1)
        newPopulation.append(p2)
        i+=2
    return newPopulation

def mutation(Population,p_m,terminalSet,functionSet):
    newPopulation=deque()
    nT=len(terminalSet)-1
    nF=len(functionSet)-1
    for p in Population:
        for i in range(len(p)):
            if random.random()>p_m:continue
            if random.random()<0.5: 
                p[i]=terminalSet[random.randint(0,nT)]
            else:
                p[i]=functionSet[random.randint(0,nF)]
        newPopulation.append(p)
    return newPopulation

In [85]:
# LOOK-UP TABLE YOU HAVE TO REPRODUCE.
nbVar = 4
dataSet=[[0,0,0,0,0],[0,0,0,1,1],[0,0,1,0,0],[0,0,1,1,0],[0,1,0,0,0],[0,1,0,1,0],[0,1,1,0,0],[0,1,1,1,1],[1,0,0,0,0],[1,0,0,1,1],[1,0,1,0,0],[1,0,1,1,0],[1,1,0,0,0],[1,1,0,1,0],[1,1,1,0,0],[1,1,1,1,0]]
print(dataSet)

cpu=CPU()

# Function and terminal sets.
functionSet=["AND", "OR", "NOT", "XOR"]
terminalSet=["X1", "X2","X3", "X4"]

# Example of program.
prog=["X1", "X2", "AND", "X3", "OR"]
progLength = 5
prog=randomProg(progLength,functionSet,terminalSet)
print(prog)

# Execute a program on one row of the data set.
data = dataSet[0]
output=execute(prog,cpu,data)
print(output)
print("-------------")

# Parameters
popSize = 100
p_c = 0.6
p_m = 0.4

# Generate the initial population
population = deque()
fittest = deque()
for i in range(popSize):
    population.append(randomProg(progLength, functionSet, terminalSet)) #add programs

# Evolution. Loop on the creation of population at generation i+1 from population at generation i, through selection, crossover and mutation.
for i in range(50):
        
    population = selection(population, cpu, dataSet)
    population = crossover(population, p_c)
    population = mutation(population, p_m, terminalSet, functionSet)

    #keep best indi
    best_fit = 0.0
    for indi in population:
        potential = computeFitness(indi, cpu, dataSet)
        if potential >= best_fit:
            best = indi
    fittest.append(best)
    population.append(best)

for p in population:
    print(p, computeFitness(p, cpu, dataSet))
    
print("----------")
    
for p in fittest:
    print(p, computeFitness(p, cpu, dataSet))

[[0, 0, 0, 0, 0], [0, 0, 0, 1, 1], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 0], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1, 1, 1, 1, 0]]
['XOR', 'AND', 'X4', 'X2', 'X4']
0
-------------
['X1', 'OR', 'XOR', 'NOT', 'NOT'] 6
['X4', 'AND', 'NOT', 'X4', 'OR'] 3
['NOT', 'X4', 'X1', 'X2', 'OR'] 5
['X3', 'XOR', 'XOR', 'OR', 'X1'] 7
['AND', 'X4', 'X4', 'NOT', 'OR'] 3
['X1', 'X2', 'X1', 'X4', 'X1'] 7
['X4', 'OR', 'NOT', 'X4', 'X1'] 7
['AND', 'X2', 'NOT', 'X4', 'XOR'] 7
['X1', 'X1', 'XOR', 'X4', 'X4'] 11
['X1', 'X4', 'X3', 'OR', 'NOT'] 9
['X2', 'XOR', 'X2', 'X4', 'X4'] 11
['NOT', 'X2', 'X2', 'X4', 'OR'] 7
['X4', 'X2', 'XOR', 'NOT', 'XOR'] 6
['OR', 'X3', 'X2', 'X2', 'X2'] 7
['X3', 'X4', 'OR', 'XOR', 'X4'] 11
['X2', 'OR', 'OR', 'X3', 'X2'] 7
['NOT', 'X1', 'X1', 'X1', 'XOR'] 13
['NOT', 'X4', 'X2', 'AND', 'NOT'] 5
['X4', 'AND', 'AND', 'XOR', 'X4'] 11
['X4', 'AND', 'X